# Table: geom

**Motivation**: KNN + Shatter dim. Device = ```cuda:1``` <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from analysis.eval import sparse_score
from figures.fighelper import *
from vae.train_vae import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Fig save dir

In [2]:
from figures.imgs import plot_weights

fig_dir = pjoin(fig_base_dir, 'nips_may22')
os.makedirs(fig_dir, exist_ok=True)
print(os.listdir(fig_dir))

kws_fig = {
    'transparent': True,
    'bbox_inches': 'tight',
    'dpi': 600,
}

[
    'phi_conv_categorical.png',
    'phi_lca.png',
    'phi_conv_laplace.png',
    'phi_conv_poisson.png',
    'phi_lin_laplace.png',
    'phi_fista.png',
    'phi_conv_gaussian.png',
    'phi_lin_poisson.png',
    'phi_lin_categorical.png',
    'nelbo_point.pdf',
    'phi_lin_gaussian.png'
]

## Device

In [3]:
device_idx = 1
device = f'cuda:{device_idx}'

from analysis.final import sort_fits, analyze_fits, add_nelbo_diff
from figures.table import table_entry_knn, table_entry_shatter
from analysis.stats import ttest

## Load dfs

In [4]:
df_knn = pd.read_pickle(pjoin(tmp_dir, 'df_knn.df'))
df_shatter = pd.read_pickle(pjoin(tmp_dir, 'df_shatter.df'))

## Shatter

### ttest

In [5]:
selected_models = [
    'poisson', 'categorical', 'laplace',
    'gaussian', 'gaussian-relu', 'gaussian-exp',
]
df_selected = df_shatter.loc[df_shatter['n_dims'].isin([10, 50, 100])].copy()
df_selected = df_selected.loc[df_selected['type'].isin(selected_models)]

In [6]:
print(df_selected['type'].unique().tolist())

['categorical', 'gaussian', 'gaussian-exp', 'gaussian-relu', 'laplace', 'poisson']

In [7]:
df_agg, df_ttest = ttest(
    value='accuracy',
    df=df_selected,
    by=['n_dims', 'type'],
    test_conds=[
        ('poisson', m) for m in
        df_selected['type'].unique()
        if m != 'poisson'],
    method='fdr_bh',
    alpha=0.01,
)

In [8]:
df_ttest

,n_dims,cond1,cond2,t,pvals,pvals_fdr_bh,reject
0,10,poisson,categorical,0.662967,5.074730e-01,5.074730e-01,False
1,10,poisson,gaussian,16.687499,1.236669e-56,1.426926e-56,True
2,10,poisson,gaussian-exp,28.113196,2.300337e-135,3.136824e-135,True
3,10,poisson,gaussian-relu,15.893258,5.624889e-52,6.026667e-52,True
4,10,poisson,laplace,19.559841,1.331953e-74,1.664942e-74,True
5,50,poisson,categorical,42.717303,3.587684e-247,5.381527e-247,True
6,50,poisson,gaussian,119.982773,0.000000e+00,0.000000e+00,True
7,50,poisson,gaussian-exp,111.222322,0.000000e+00,0.000000e+00,True
8,50,poisson,gaussian-relu,109.848941,0.000000e+00,0.000000e+00,True
9,50,poisson,laplace,100.845083,0.000000e+00,0.000000e+00,True


In [9]:
df_ttest.pivot(index='cond2', columns='n_dims', values='reject')

n_dims,10,50,100
cond2,,,
categorical,False,True,True
gaussian,True,True,True
gaussian-exp,True,True,True
gaussian-relu,True,True,True
laplace,True,True,True


In [10]:
for index, values in df_agg.items():
    print(index, len(values))
    break

(10, 'categorical')
1260

### ci

In [11]:
%%time

for dim in [10, 50, 100]:
    table = table_entry_shatter(dim, df_agg, n_resamples=int(1e5), fmt='0.3f')
    print(f"——— dim: {dim} ———\n")
    print(table)
    print('\n\n\n')

——— dim: 10 ———

\begin{tabular}{c}
    \entry{0.797}{.009} \\ 
    \entry{0.795}{.006} \\ 
    \entry{0.751}{.008} \\ 
    \entry{0.758}{.007} \\ 
    \entry{0.762}{.007} \\ 
    \entry{0.737}{.008}
\end{tabular}

——— dim: 50 ———

\begin{tabular}{c}
    \entry{0.935}{.003} \\ 
    \entry{0.899}{.004} \\ 
    \entry{0.770}{.007} \\ 
    \entry{0.743}{.007} \\ 
    \entry{0.786}{.006} \\ 
    \entry{0.762}{.007}
\end{tabular}

——— dim: 100 ———

\begin{tabular}{c}
    \entry{0.949}{.002} \\ 
    \entry{0.884}{.004} \\ 
    \entry{0.767}{.007} \\ 
    \entry{0.763}{.007} \\ 
    \entry{0.818}{.006} \\ 
    \entry{0.793}{.006}
\end{tabular}

CPU times: user 39.2 s, sys: 9.67 s, total: 48.9 s
Wall time: 48.9 s


## KNN

### ttest

In [12]:
selected_models = [
    'poisson', 'categorical', 'laplace',
    'gaussian', 'gaussian-relu', 'gaussian-exp',
]
df_selected = df_knn.loc[df_knn['n_dims'].isin([10, 50, 100])].copy()
df_selected = df_selected.loc[df_selected['type'].isin(selected_models)]

In [13]:
print(df_selected['type'].unique().tolist())

['categorical', 'gaussian', 'gaussian-exp', 'gaussian-relu', 'laplace', 'poisson']

In [14]:
df_agg, df_ttest = ttest(
    value='accuracy',
    df=df_selected,
    by=['n_dims', 'size', 'type'],
    test_conds=[
        ('poisson', m) for m in
        df_selected['type'].unique()
        if m != 'poisson'],
    method='fdr_bh',
    alpha=0.01,
)

In [15]:
df_ttest

,n_dims,size,cond1,cond2,t,pvals,pvals_fdr_bh,reject
0,10,200,poisson,categorical,169.881202,0.000000,0.000000,True
1,10,200,poisson,gaussian,182.398750,0.000000,0.000000,True
2,10,200,poisson,gaussian-exp,165.188618,0.000000,0.000000,True
3,10,200,poisson,gaussian-relu,145.069923,0.000000,0.000000,True
4,10,200,poisson,laplace,82.033128,0.000000,0.000000,True
5,10,1000,poisson,categorical,240.189356,0.000000,0.000000,True
6,10,1000,poisson,gaussian,221.416368,0.000000,0.000000,True
7,10,1000,poisson,gaussian-exp,203.872618,0.000000,0.000000,True
8,10,1000,poisson,gaussian-relu,161.395089,0.000000,0.000000,True
9,10,1000,poisson,laplace,132.399649,0.000000,0.000000,True


In [16]:
df_ttest.pivot(index=['n_dims', 'cond2'], columns='size', values='reject')

size                  200   1000   5000
n_dims cond2                           
10     categorical    True  True   True
       gaussian       True  True   True
       gaussian-exp   True  True   True
       gaussian-relu  True  True   True
       laplace        True  True  False
50     categorical    True  True   True
       gaussian       True  True   True
       gaussian-exp   True  True   True
       gaussian-relu  True  True   True
       laplace        True  True   True
100    categorical    True  True   True
       gaussian       True  True   True
       gaussian-exp   True  True   True
       gaussian-relu  True  True  False
       laplace        True  True  False

In [17]:
for index, values in df_agg.items():
    print(index, len(values))
    break

(10, 200, 'categorical')
2500

### ci=0.99

In [18]:
%%time

for dim in [10, 50, 100]:
    table = table_entry_knn(dim, df_agg, n_resamples=int(1e5), fmt='0.3f')
    print(f"——— dim: {dim} ———\n")
    print(table)
    print('\n\n\n')

——— dim: 10 ———

\begin{tabularx}{65mm}{CCC}
    \entry{0.815}{.002} & \entry{0.919}{.001} & \entry{0.946}{.017} \\ 
    \entry{0.705}{.002} & \entry{0.800}{.002} & \entry{0.853}{.040} \\ 
    \entry{0.757}{.003} & \entry{0.869}{.002} & \entry{0.924}{.028} \\ 
    \entry{0.673}{.003} & \entry{0.813}{.002} & \entry{0.891}{.033} \\ 
    \entry{0.694}{.003} & \entry{0.817}{.003} & \entry{0.877}{.045} \\ 
    \entry{0.642}{.003} & \entry{0.784}{.002} & \entry{0.863}{.032}
\end{tabularx}

——— dim: 50 ———

\begin{tabularx}{65mm}{CCC}
    \entry{0.825}{.002} & \entry{0.927}{.001} & \entry{0.957}{.005} \\ 
    \entry{0.770}{.002} & \entry{0.880}{.001} & \entry{0.920}{.009} \\ 
    \entry{0.710}{.003} & \entry{0.836}{.003} & \entry{0.902}{.038} \\ 
    \entry{0.604}{.003} & \entry{0.746}{.002} & \entry{0.837}{.022} \\ 
    \entry{0.710}{.002} & \entry{0.844}{.002} & \entry{0.904}{.026} \\ 
    \entry{0.694}{.003} & \entry{0.836}{.002} & \entry{0.906}{.027}
\end{tabularx}

——— dim: 100 ———

\begin{tabularx}{65mm}{CCC}
    \entry{0.807}{.002} & \entry{0.925}{.001} & \entry{0.958}{.013} \\ 
    \entry{0.753}{.002} & \entry{0.876}{.001} & \entry{0.925}{.005} \\ 
    \entry{0.701}{.004} & \entry{0.830}{.003} & \entry{0.896}{.046} \\ 
    \entry{0.636}{.003} & \entry{0.789}{.002} & \entry{0.875}{.024} \\ 
    \entry{0.757}{.002} & \entry{0.881}{.001} & \entry{0.933}{.019} \\ 
    \entry{0.695}{.003} & \entry{0.846}{.002} & \entry{0.918}{.024}
\end{tabularx}

CPU times: user 2min 37s, sys: 37.6 s, total: 3min 14s
Wall time: 3min 14s
